In [2]:
#imports
import numpy as np
from math import sqrt
from numpy import concatenate

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

from matplotlib import pyplot
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

import keras.utils
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

import tensorflow as tf
import seaborn as sn
import seed
import os
tf.get_logger().setLevel('ERROR')


In [3]:
def create_lagged_features(data, to_keep=1, to_remove=1):
    variables = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    columns, names = list(), list()

    for i in range(to_keep, 0, -1):
        columns.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(variables)]

    for i in range(0, to_remove):
        columns.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(variables)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(variables)]

    #put it all together
    final = concat(columns, axis=1)
    final.columns = names

    #drop rows with NaN values
    final.dropna(inplace=True)

    return final


In [4]:
def rsi(data, period: int = 14):

    delta = data["Close"].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    gain = up.ewm(com=(period - 1), min_periods=period).mean()
    loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()

    RS = gain / loss
    return 100 - (100 / (1 + RS))


In [5]:
def load_lag_sets(step, lag_features, train_ratio, lag_granularity, lag, dataset_grouped_by, cleaned):

    analyser = "vader"

    #folder to read
    folder = "./../../datasets/tweets_prices_volumes_sentiment/" + \
        analyser+"/"+dataset_grouped_by+"_datasets"
    #add cleaned if to use cleaned data
    if cleaned:
        folder = folder + '/cleaned'
    #get full filename
    filename = folder+"/final_data_lag_"+lag_granularity+"_" + \
        str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

    #read dataset
    df = pd.read_csv(filename)

    #group by datetime
    df = df.groupby('DateTime').agg(lambda x: x.mean())

    #calculate price direction
    df["Change"] = (df["Close"] > df["Close"].shift(1)).astype(int)

    #whether to add RSI, long and short moving averages
    add_RSI = False
    add_longMAvg = False
    add_shortMAvg = False

    if(add_RSI):
        #calculate RSI
        RSI = 14
        df['RSI'] = rsi(df, RSI)
        df = df.iloc[RSI:]

    #calcualte moving averages
    if(add_shortMAvg):
        short_window = 9
        df['short_mavg'] = df.rolling(window=short_window)["Close"].mean()

    if(add_longMAvg):
        long_window = 21
        df["long_mavg"] = df.rolling(window=long_window)["Close"].mean()

    if(add_longMAvg):
        df = df.iloc[long_window:]
    elif(add_RSI):
        df = df.iloc[RSI:]
    elif(add_shortMAvg):
        df = df.iloc[short_window:]

    #keep only wanted columns
    features = ['Change', 'subjectivity', 'polarity', 'Tweet_vol', 'Volume_(BTC)'] if analyser == "Textblob" else [
        'Change', 'Close', 'pos_pol', 'neg_pol', 'Tweet_vol']

    if(add_RSI):
        features.append("RSI")

    if(add_longMAvg):
        features.append("long_mavg")

    if(add_shortMAvg):
        features.append("short_mavg")

    #keep only wanted features
    df = df[features]

    #creating copy so that data is not loaded once again
    df_copy = df.copy()

    #number of previous records to consider for every example
    n_lag = lag_features
    #number of features
    n_features = len(features)
    #calculate total_features
    total_features = n_lag*n_features

    if(total_features == 0):
        total_features = n_features

    #add lagged features
    df_with_lagged = create_lagged_features(df, n_lag, 1)
    df_with_lagged = df_with_lagged.reset_index()
    df_with_lagged = df_with_lagged.drop(['DateTime'], axis=1)

    #shuffle
    np.random.seed(1)
    for j in range(0, step+1):
        df_with_lagged = shuffle(df_with_lagged)

    #divide df into train and test
    data_len = len(df_with_lagged)
    train_size = int(data_len*train_ratio)

    #get training data
    train = df_with_lagged.iloc[:train_size]
    #get testing data
    test = df_with_lagged.iloc[train_size:]

    #get labels
    train_y = train["var1(t)"].values
    test_y = test["var1(t)"].values

    #reshape labels
    train_y = train_y.reshape(len(train_y), 1)
    test_y = test_y.reshape(len(test_y), 1)

    #normalise features
    xscaler = MinMaxScaler(feature_range=(0, 1))
    train = xscaler.fit_transform(train)
    test = xscaler.transform(test)

    train_labels = train_y
    test_labels = test_y

    #remove the last set of values(data of time to be predicted)
    train = train[:, :total_features]
    test = test[:, :total_features]

    #keep only prices array
    train_X, train_y = train[:, :total_features], train_y[:, -1]
    test_X, test_y = test[:, :total_features], test_y[:, -1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_lag, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_lag, n_features))

    #get labels as categorical
    train_y = keras.utils.to_categorical(train_y, 2)

    return train_X, test_X, train_y, test_y, len(features), df, train_size


In [6]:
def create_model_test(epochs, neurons, batch_size, layers, train_X, test_X, train_y, test_y, lag_features, features, df, train_size):

    #set seed to reproduce results
    np.random.seed(1)
    tf.random.set_seed(1)

    # design network
    model = Sequential()
    dropout = 0.25
    activ_func = "linear"

    #return sequences flag if there are more than 1 layer
    return_seq = layers > 1

    #add first layer
    model.add(LSTM(neurons, return_sequences=return_seq, input_shape=(
        train_X.shape[1], train_X.shape[2]), activation=activ_func))
    model.add(Dropout(dropout))

    #add the other layers
    for i in range(1, layers):
        ret_seq = i != (layers-1)
        model.add(LSTM(neurons, return_sequences=ret_seq, activation=activ_func))
        model.add(Dropout(dropout))

    #add a dense layer to output the prediction
    model.add(Dense(2, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    # fit network
    history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                        verbose=0, shuffle=False, validation_split=0.2, callbacks=[callback])

    #reshape
    test_X = test_X.reshape((test_X.shape[0], lag_features, features))

    #make prediction
    pred = model.predict(test_X)

    #reshape again
    test_X = test_X.reshape((test_X.shape[0], lag_features * features,))

    #get prediction
    pred = np.argmax(pred, axis=1)

    #calculate accuracy
    prices = pd.DataFrame()
    prices["Actual"] = test_y
    prices["Predicted"] = pred
    prices["Correct"] = (prices["Actual"] - prices["Predicted"]) == 0
    incorrect = prices.loc[prices['Correct'] == False]
    incorrect_len = len(incorrect)
    prices_len = len(prices)
    accuracy = ((prices_len-incorrect_len)/prices_len)

    return accuracy


In [15]:
def implement(lag_granularity, lag, dataset_grouped_by, cleaned):
    #build result filename
    filename = '../results/lstm_trend2/lstm_groupedby_' + \
        dataset_grouped_by+"_lag_"+lag_granularity+"_"+str(lag)
    if cleaned:
        filename = filename + '_cleaned'
    full_filename = filename+".csv"

    #columns for csv result file
    columns = ["lag", "batch_size", "neurons", "layers",
               "split", "mean_acc", "min_acc", "max_acc", "diff_acc"]

    #try to read data if the result file exists, otherwise create a new dataframe
    try:
        results = pd.read_csv(full_filename)
    except:
        results = pd.DataFrame(columns=columns)

    #lagged_features
    lags = [2, 4, 5]
    #train_ratio
    train_ratio = 0.85

    #for each lag feature
    for lag_features in lags:

        #combinations
        epochs = [10000]
        neurons = [16, 32]
        layers = [1, 2, 3]
        batch_sizes = [5, 20, 50]

        #for each epoch, neuron, layers and batch_size value
        for n in neurons:
            for l in layers:
                for b in batch_sizes:

                    accuracies = []
                    print("Testing model: lag:", lag_features,
                          ", neurons:", n, ", layers:", l, ", batch_size:", b)

                    #run for 5 times
                    for i in range(0, 5):
                        train_X, test_X, train_y, test_y, features, df, train_size = load_lag_sets(
                            i, lag_features, train_ratio, lag_granularity, lag, dataset_grouped_by, cleaned)
                        acc = create_model_test(
                            10000, n, b, l, train_X, test_X, train_y, test_y, lag_features, features, df, train_size)
                        accuracies.append(acc)

                    #calculate mean values
                    accuracies = np.array(accuracies)
                    mean_acc = accuracies.mean()
                    min_acc = accuracies.min()
                    max_acc = accuracies.max()
                    diff_acc = max_acc - min_acc

                    #add to df
                    results = results.append({"lag": lag_features, "batch_size": b, "neurons": n, "layers": l, "split": train_ratio,
                                             "mean_acc": mean_acc, "min_acc": min_acc, "max_acc": max_acc, "diff_acc": diff_acc}, ignore_index=True)

    return results, full_filename


In [16]:
#lag granularity - days or hours
lag_granularity = "days"
#lag value
lag = 3
#dataset grouped type - day or hour
dataset_grouped_by = "day"
#cleaned
cleaned = True


In [17]:
results, full_filename = implement(lag_granularity, lag, dataset_grouped_by, cleaned)

Testing model: lag: 2 , neurons: 16 , layers: 1 , batch_size: 5


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_21144\647389347.py:19: FutureWarning: ['dt'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df = df.groupby('DateTime').agg(lambda x: x.mean())


3/3 [==============================] - 0s 3ms/step


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_21144\647389347.py:19: FutureWarning: ['dt'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df = df.groupby('DateTime').agg(lambda x: x.mean())


3/3 [==============================] - 0s 2ms/step


...

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_21144\647389347.py:19: FutureWarning: ['dt'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df = df.groupby('DateTime').agg(lambda x: x.mean())


3/3 [==============================] - 0s 4ms/step


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_21144\647389347.py:19: FutureWarning: ['dt'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df = df.groupby('DateTime').agg(lambda x: x.mean())


3/3 [==============================] - 0s 4ms/step


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_21144\2372472743.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"lag": lag_features, "batch_size": b, "neurons": n, "layers": l, "split": train_ratio,


In [18]:
results


,lag,batch_size,neurons,layers,split,mean_acc,min_acc,max_acc,diff_acc
0,2.0,5.0,16.0,1.0,0.85,0.523529,0.411765,0.617647,0.205882
1,2.0,20.0,16.0,1.0,0.85,0.511765,0.397059,0.588235,0.191176
2,2.0,50.0,16.0,1.0,0.85,0.508824,0.367647,0.588235,0.220588
3,2.0,5.0,16.0,2.0,0.85,0.494118,0.397059,0.558824,0.161765
4,2.0,20.0,16.0,2.0,0.85,0.520588,0.382353,0.617647,0.235294
...,...,...,...,...,...,...,...,...,...
67,5.0,20.0,32.0,2.0,0.85,0.541176,0.500000,0.573529,0.073529
68,5.0,50.0,32.0,2.0,0.85,0.514706,0.470588,0.558824,0.088235
69,5.0,5.0,32.0,3.0,0.85,0.550000,0.485294,0.617647,0.132353
70,5.0,20.0,32.0,3.0,0.85,0.535294,0.441176,0.617647,0.176471


In [19]:
#create folder if it does not exist
folder = full_filename.rsplit('/', 1)[0]
if not os.path.exists(full_filename):
    os.makedirs(full_filename)


In [20]:
results.to_csv(full_filename, index=False)
